In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from my_functions import *

In [ ]:
tcurves = load_tcurves(load_filter_tags())
w_central = central_wavelength(tcurves)

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_1sq.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

qso_flx = []
qso_err = []
zspec = []
with open(filename) as f:
    rdr = csv.reader(f, delimiter=' ')
    next(rdr, None)
    for row in rdr:
        qso_flx.append(row[2 : 2 + 60])
        qso_err.append(row[2 + 60 : 2 + 60 + 60])
        zspec.append(row[-1])
qso_flx = np.array(qso_flx).T[my_filter_order].astype(float)
qso_err = np.array(qso_err).T[my_filter_order].astype(float)
zspec = np.array(zspec).astype(float)

In [ ]:
N_sources = qso_flx.shape[1]
N_nb = 5
ew0min =0


In [ ]:
nb_c_min = 6
nb_c_max = 50
line_qso = np.zeros((nb_c_max - nb_c_min, N_sources)).astype(bool)
cont_est_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
cont_err_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
i = 0
for nb_c in range(nb_c_min, nb_c_max):
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(nb_c, tcurves=tcurves)
    cont_est_qso, cont_err_qso = stack_estimation(qso_flx, qso_err, nb_c, N_nb, False)
    line_qso[i] = (
        (qso_flx[nb_c] - cont_est_qso > 3 * (cont_err_qso**2 + qso_err[nb_c]**2) ** 0.5)
        & (qso_flx[nb_c] - cont_est_qso > ew0min * (1 + z_nb) * cont_est_qso / fwhm)
    )
    cont_est_Arr[i] = cont_est_qso
    cont_err_Arr[i] = cont_err_qso
    i += 1

In [ ]:
j = 0
for mock_n in np.where(zspec > 0)[0]:
    fig = plt.figure(figsize=(10,8))
    ax = plot_JPAS_source(qso_flx[:, mock_n], qso_err[:, mock_n])

    for nb in np.where(line_qso[:, mock_n])[0]:
        idx = nb + nb_c_min
        ax.errorbar(w_central[idx], cont_est_Arr[nb, mock_n], fmt='^', c='C4',
            markersize=10, yerr=cont_err_Arr[nb, mock_n], capsize=3,
            capthick=3, elinewidth=4)
    plt.show()

    try: first = np.where(line_qso[:, mock_n])[0][0] + nb_c_min
    except: continue
    z_nb = w_central[first] / 1215.67 - 1

    print('z spec = {0:0.2f}'.format(zspec[mock_n]))
    print('z NB   = {0:0.2f}'.format(z_nb))

    j += 1
    if j == 10: break

In [ ]:
# Arr where the center of the Lya line is. -1 means no lines detected
line_lya_center = np.ones(N_sources, dtype=int) * -1

for mock_n in range(N_sources):
    try:
        first = np.where(line_qso[:, mock_n])[0][0] # First True in line_qso
    except:
        continue
    if first > 40: # We don't expect lines past here
        continue

    line_nb_width = 0
    while line_qso[first + line_nb_width + 1, mock_n]:
        line_nb_width += 1

    line_lya_center[mock_n] = np.argmax(
        qso_flx[first + nb_c_min : first + nb_c_min + line_nb_width + 1, mock_n]
    ) + first + nb_c_min
print(len(np.where((line_lya_center != -1))[0]))

In [ ]:
line_CIV = np.zeros(N_sources, dtype=bool)
nb_CIV_Arr = np.ones(N_sources, dtype=int) * -1
for mock_n in np.where(line_lya_center != -1)[0]:
    w_obs_lya = w_central[line_lya_center[mock_n]]
    z = w_obs_lya / 1215.67 - 1
    w_obs_CIV = 1549.48 * (1 + z)

    if w_obs_CIV > 8500.: continue # Out of range

    nb_CIV_Arr[mock_n] = np.argmin(np.abs(w_central[:-4] - w_obs_CIV))
    nb_CIV = nb_CIV_Arr[mock_n]
    line_CIV[mock_n] = qso_flx[nb_CIV, mock_n] - cont_est_Arr[nb_CIV - nb_c_min, mock_n]\
        > 3 * (cont_err_Arr[nb_CIV - nb_c_min, mock_n]**2 + qso_err[nb_CIV, mock_n]**2)\
            ** 0.5
print(len(np.where(line_CIV)[0]))

In [ ]:
## CIV line
j = 0
for mock_n in np.where(line_CIV & (zspec > 2.5))[0]:
    fig = plt.figure(figsize=(10, 8))
    ax = plot_JPAS_source(qso_flx[:, mock_n], qso_err[:, mock_n])

    ax.errorbar(w_central[line_lya_center[mock_n]],
        cont_est_Arr[line_lya_center[mock_n], mock_n],
        yerr=cont_err_Arr[line_lya_center[mock_n], mock_n],
        fmt='^', c='C4', markersize=10, capsize=3, capthick=3, elinewidth=4)

    ax.errorbar(w_central[nb_CIV_Arr[mock_n]],
        cont_est_Arr[nb_CIV_Arr[mock_n], mock_n],
        yerr=cont_err_Arr[nb_CIV_Arr[mock_n], mock_n],
        fmt='^', c='C6', markersize=10, capsize=3, capthick=3, elinewidth=4)

    print('z_spec = {0:0.2f}'.format(zspec[mock_n]))
    print('z NB   = {0:0.2f}'.format(w_central[line_lya_center[mock_n]] / 1215.67 - 1))

    plt.show()

    j += 1
    if j == 10: break

In [ ]:
line_CIII = np.zeros(N_sources, dtype=bool)
nb_CIII_Arr = np.ones(N_sources, dtype=int) * -1
for mock_n in np.where(line_CIV)[0]:
    w_obs_lya = w_central[line_lya_center[mock_n]]
    z = w_obs_lya / 1215.67 - 1
    w_obs_CIII = 1908.73 * (1 + z)

    if w_obs_CIII > 8200.: continue # Out of range

    nb_CIII_Arr[mock_n] = np.argmin(np.abs(w_central[:-4] - w_obs_CIII))
    nb_CIII = nb_CIII_Arr[mock_n]
    line_CIII[mock_n] = qso_flx[nb_CIII, mock_n] - cont_est_Arr[nb_CIII - nb_c_min, mock_n]\
        > 3 * (cont_err_Arr[nb_CIII - nb_c_min, mock_n]**2 + qso_err[nb_CIII, mock_n]**2)\
            ** 0.5
print(len(np.where(line_CIII)[0]))

In [ ]:
## CIII] line
j = 0
for mock_n in np.where(line_CIII & (zspec > 2.5))[0]:
    fig = plt.figure(figsize=(10, 8))
    ax = plot_JPAS_source(qso_flx[:, mock_n], qso_err[:, mock_n])

    ax.errorbar(w_central[line_lya_center[mock_n]],
        cont_est_Arr[line_lya_center[mock_n], mock_n],
        yerr=cont_err_Arr[line_lya_center[mock_n], mock_n],
        fmt='^', c='C4', markersize=10, capsize=3, capthick=3, elinewidth=4)

    ax.errorbar(w_central[nb_CIV_Arr[mock_n]],
        cont_est_Arr[nb_CIV_Arr[mock_n], mock_n],
        yerr=cont_err_Arr[nb_CIV_Arr[mock_n], mock_n],
        fmt='^', c='C6', markersize=10, capsize=3, capthick=3, elinewidth=4)

    ax.errorbar(w_central[nb_CIII_Arr[mock_n]],
        cont_est_Arr[nb_CIII_Arr[mock_n], mock_n],
        yerr=cont_err_Arr[nb_CIII_Arr[mock_n], mock_n],
        fmt='^', c='C7', markersize=10, capsize=3, capthick=3, elinewidth=4)

    print('z_spec = {0:0.2f}'.format(zspec[mock_n]))

    plt.show()

    j += 1
    if j == 10: break